# Calculadora de avance

## Datos del proyecto

In [1]:
proyecto = "sibayo"
mes_en_analisis = 5
anio = 2025

## Librerias necesarias

In [2]:
from dotenv import load_dotenv, find_dotenv
from dotenv import load_dotenv, find_dotenv
from functools import reduce
from google.cloud import firestore
from google.oauth2 import service_account
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from pathlib import Path
from typing import Dict, List, Any
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import excel2img
import itertools
import json
import os
import pandas as pd
import pickle
import pprint
import re
import win32com.client as win32
import xlsxwriter

## Funciones utiles

### Fusionar diccionarios

In [3]:
def fusionar_diccionarios(diccionario_de_diccionarios):
    """
    Fusiona diccionarios separados en un diccionario unificado.
    Solo incluye claves donde ambos valores sean diferentes de cero.
    Usa dinámicamente las claves del diccionario de entrada.
    
    Args:
        diccionario_de_diccionarios (dict): Diccionario con estructura:
            {
                "nombre_campo1": {clave: valor, ...},
                "nombre_campo2": {clave: valor, ...}
            }
    
    Returns:
        dict: Diccionario fusionado con estructura {clave: {nombre_campo1: valor, nombre_campo2: valor}}
    """
    # Obtener las claves (nombres de los campos) del diccionario principal
    nombres_campos = list(diccionario_de_diccionarios.keys())
    
    if len(nombres_campos) != 2:
        raise ValueError("El diccionario debe contener exactamente 2 campos")
    
    campo1_nombre = nombres_campos[0]
    campo2_nombre = nombres_campos[1]
    
    campo1_datos = diccionario_de_diccionarios[campo1_nombre]
    campo2_datos = diccionario_de_diccionarios[campo2_nombre]
    
    fusionado = {}
    
    # Obtener todas las claves únicas de ambos diccionarios
    todas_las_claves = set(campo1_datos.keys()) | set(campo2_datos.keys())
    
    for clave in todas_las_claves:
        valor1 = campo1_datos.get(clave, 0)
        valor2 = campo2_datos.get(clave, 0)
        
        # Solo agregar si ambos valores son diferentes de cero
        if valor1 != 0 and valor2 != 0:
            fusionado[clave] = {
                campo1_nombre: valor1,
                campo2_nombre: valor2
            }
    
    return fusionado

### Calculadora de costo total

In [4]:
from typing import TypedDict


class CostoIngenieriaResult(TypedDict):
    costo_directo: float
    gastos_generales: float
    utilidad: float
    subtotal: float
    igv: float
    total: float


def calculadora_costo_total(costo_directo: float, gastos_generales: float) -> CostoIngenieriaResult:
    """
    Calcula el costo total de un proyecto de ingeniería civil en soles peruanos.

    Args:
        costo_directo (float): Costo directo del proyecto

    Returns:
        CostoIngenieriaResult: Diccionario con todos los valores calculados
    """
    # Calcular gastos generales (10% del costo directo)
    gastos_generales = round(costo_directo * gastos_generales, 2)

    # Calcular utilidad (5% del costo directo)
    utilidad = round(costo_directo * 0.05, 2)

    # Calcular subtotal
    subtotal = round(costo_directo + gastos_generales + utilidad, 2)

    # Calcular IGV (18% del subtotal)
    igv = round(subtotal * 0.18, 2)

    # Calcular total
    total = round(subtotal + igv, 2)

    return {
        "costo_directo": round(costo_directo, 2),
        "gastos_generales": gastos_generales,
        "utilidad": utilidad,
        "subtotal": subtotal,
        "igv": igv,
        "total": total,
    }

### Multiplicador de precio unitario actualizado por cargas de trabajo

In [5]:
def calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Si alguna clave en cargas_trabajo tiene carga > 0 y no existe
    en dict_precios_unitarios_actualizados, levanta KeyError.
    Claves con carga = 0 se ignoran (no dan error), y claves en precios que noprecios_uninatrios_expediente
    están en cargas se tratan con carga = 0.

    Args:
        dict_precios_unitarios_actualizados (dict): código → precio unitario.
        cargas_trabajo (dict): código → carga de trabajo.

    Returns:
        dict: código → precio total (precio_unitario * carga).

    Raises:
        KeyError: si alguna clave con carga > 0 en cargas_trabajo
                  no existe en dict_precios_unitarios_actualizados.
    """
    # Detectar faltantes solo para cargas > 0
    faltantes = {
        codigo
        for codigo, carga in cargas_trabajo.items()
        if carga != 0 and codigo not in dict_precios_unitarios_actualizados
    }
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Construir resultado: para cada precio, multiplicar por carga (0 si no hay)
    resultado = {}
    for codigo, precio_unitario in dict_precios_unitarios_actualizados.items():
        carga = cargas_trabajo.get(codigo, 0)
        resultado[codigo] = precio_unitario * carga

    return resultado

### Multiplicador de precio unitario por carga de trabajo v2

In [6]:
def calcular_costos_unitarios_v2(
    dict_precios_unitarios_actualizados, cargas_trabajo
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Si alguna clave en cargas_trabajo tiene carga > 0 y no existe
    en dict_precios_unitarios_actualizados, levanta KeyError.
    Claves con carga = 0 se ignoran (no dan error), y claves en precios que no
    están en cargas se tratan con carga = 0.

    Args:
        dict_precios_unitarios_actualizados (dict): código → precio unitario.
        cargas_trabajo (dict): código → carga de trabajo.

    Returns:
        dict: código → precio total (precio_unitario * carga).

    Raises:
        KeyError: si alguna clave con carga > 0 en cargas_trabajo
                  no existe en dict_precios_unitarios_actualizados.
    """
    # Detectar faltantes solo para cargas > 0
    faltantes = {
        codigo
        for codigo, carga in cargas_trabajo.items()
        if carga != 0 and codigo not in dict_precios_unitarios_actualizados
    }
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Construir resultado: para cada precio, multiplicar por carga (0 si no hay)
    resultado = {}
    for codigo, precio_unitario in dict_precios_unitarios_actualizados.items():
        carga = cargas_trabajo.get(codigo, 0)
        resultado[codigo] = precio_unitario * carga

    return resultado

### Sumador de diccionarios

In [7]:
from typing import Any, Dict, Union

def sumar_dicts(*dicts: Dict[Any, Any]) -> Dict[Any, Union[int, float]]:
    """
    Suma múltiples diccionarios numéricos.

    Args:
        *dicts: Diccionarios con valores numéricos (int o float).

    Returns:
        Un nuevo diccionario con la suma de valores por clave.

    Raises:
        TypeError: si algún valor no es int ni float o un argumento no es dict.
    """
    result: Dict[Any, Union[int, float]] = {}

    for idx, d in enumerate(dicts, start=1):
        if not isinstance(d, dict):
            raise TypeError(f"El argumento en posición {idx} no es un dict.")
        for key, value in d.items():
            if not isinstance(value, (int, float)):
                raise TypeError(
                    f"El valor para clave '{key}' en el dict {idx} "
                    "no es numérico (int o float)."
                )
            result[key] = result.get(key, 0) + value

    return result

## Carga de datos

### Pickle

In [8]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    cargas_trabajo_programadas_anualmente = pickle.load(f)


In [9]:
#eliminando el total
cargas_trabajo_programadas_anualmente = cargas_trabajo_programadas_anualmente.iloc[:-1]

cargas_trabajo_programadas_anualmente.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR112,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR201,862.344000,5146.590909,1470.454545,2940.909091,2940.909091,1470.454545,8822.727273,5019.472800,3676.136364,32350.00
MR202,4.078824,5.098529,0.000000,2.039412,3.059118,4.078824,6.118235,8.157647,2.039412,34.67
MR203,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR301,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,0.000000,15999.00
MR401,16.167500,0.000000,0.000000,0.000000,0.000000,16.167500,16.167500,16.167500,0.000000,64.67


### Firebase

In [10]:
## Datos de # 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("No se encontró ningún archivo .env en este directorio ni en sus padres.")
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [11]:
db=firestore.Client(credentials=credentials, project=credentials.project_id)

#### Cargas de trabajo

In [12]:
cargas_trabajo_contratista=db.collection("rutinarios").document(proyecto).collection("valorizaciones").document(str(mes_en_analisis)).get().to_dict()

pprint.pprint(cargas_trabajo_contratista)

{'cargas_trabajo_contratista_corregido': {'MR101': 0.86,
                                          'MR103': 13.33,
                                          'MR201': 5146.67,
                                          'MR202': 5.1,
                                          'MR301': 1999.88,
                                          'VP101': 242.474,
                                          'VP102': 112},
 'cargas_trabajo_contratista_inicial': {'MR101': 0.86,
                                        'MR103': 13.33,
                                        'MR201': 5146.67,
                                        'MR202': 5.1,
                                        'MR301': 1999.88,
                                        'VP101': 242.5,
                                        'VP102': 112}}


In [13]:
cargas_trabajo_contratista_corregido=cargas_trabajo_contratista['cargas_trabajo_contratista_corregido']

#### Expediente técnico

In [14]:
doc_proyecto_firebase = db.collection("rutinarios").document(proyecto).get().to_dict()
expediente_tecnico_firebase = doc_proyecto_firebase["expediente"]

pprint.pprint(expediente_tecnico_firebase)

{'cargas_trabajo': {'MR101': 10.28,
                    'MR103': 53.33,
                    'MR104': 85.33,
                    'MR201': 32350,
                    'MR202': 34.67,
                    'MR301': 15999,
                    'MR401': 64.67,
                    'MR601': 258.02,
                    'VP101': 1512.58,
                    'VP102': 700},
 'codigo_ruta': 'AR-683',
 'coordenadas': {'fin': {'altitud': 3629,
                         'datum': 'WGS84',
                         'hemisferio': 'S',
                         'progresiva': 32252.0,
                         'x': 220383.97,
                         'y': 8270075.15,
                         'zona': None,
                         'zona_letra': None},
                 'inicio': {'altitud': 3820,
                            'datum': 'WGS84',
                            'hemisferio': 'S',
                            'progresiva': 0,
                            'x': 226316.6,
                            'y': 8281146.

In [15]:
progresiva_inicio = expediente_tecnico_firebase["coordenadas"][
    "inicio"
]["progresiva"]
progresiva_fin = expediente_tecnico_firebase["coordenadas"][
    "fin"
]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
32252.0


In [16]:
gastos_generales_expediente=expediente_tecnico_firebase['gastos_generales']
print(gastos_generales_expediente)

0.1


#### Precios unitarios

In [17]:
precios_unitarios_expediente=expediente_tecnico_firebase["precios_unitarios"]
pprint.pprint(precios_unitarios_expediente)

{'MR101': 317.8,
 'MR103': 19.07,
 'MR104': 21.18,
 'MR201': 0.53,
 'MR202': 95.34,
 'MR301': 0.17,
 'MR401': 13.28,
 'MR601': 2.44,
 'VP101': 47.01,
 'VP102': 71.79}


In [18]:
cargas_trabajo_expediente=expediente_tecnico_firebase["cargas_trabajo"]

In [19]:
costos_expediente=fusionar_diccionarios({
    'carga_trabajo':cargas_trabajo_expediente,
    'precio_unitario':precios_unitarios_expediente
})

pprint.pprint(costos_expediente)

{'MR101': {'carga_trabajo': 10.28, 'precio_unitario': 317.8},
 'MR103': {'carga_trabajo': 53.33, 'precio_unitario': 19.07},
 'MR104': {'carga_trabajo': 85.33, 'precio_unitario': 21.18},
 'MR201': {'carga_trabajo': 32350, 'precio_unitario': 0.53},
 'MR202': {'carga_trabajo': 34.67, 'precio_unitario': 95.34},
 'MR301': {'carga_trabajo': 15999, 'precio_unitario': 0.17},
 'MR401': {'carga_trabajo': 64.67, 'precio_unitario': 13.28},
 'MR601': {'carga_trabajo': 258.02, 'precio_unitario': 2.44},
 'VP101': {'carga_trabajo': 1512.58, 'precio_unitario': 47.01},
 'VP102': {'carga_trabajo': 700, 'precio_unitario': 71.79}}


#### Contrato

In [20]:
contrato = doc_proyecto_firebase["contrato"]
pprint.pprint(contrato)

{'contratista': {'razon_social': 'PLATERS MANAGEMENT S.A.C.', 'ruc': ''},
 'denominacion_tramo_convenio': 'EMP. AR-111 (NUEVO SIBAYO) - TUTI EMP. AR-681 '
                                '(DV. CHIVAY) (KM 32+252)',
 'fecha_inicio': {'anio': 2025, 'dia': 15, 'mes': 4},
 'id_contrato': '004-2025',
 'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                        'dni': 0,
                        'nombre': 'Genaro',
                        'titulo': 'Ingeniero'},
 'monto_contrato': 165130.4,
 'numero_cuadrillas': 1,
 'numero_trabajadores': 3,
 'tiempo_ejecucion_dias': 240,
 'tipo_servicio': 'mantenimiento rutinario'}


In [21]:
monto_contrato = contrato["monto_contrato"]
print(monto_contrato)

165130.4


#### Valorizaciones

In [22]:
valorizaciones_ref = db.collection("rutinarios").document(proyecto).collection('valorizaciones')

pprint.pprint(valorizaciones_ref)

In [23]:
docs = valorizaciones_ref.stream()

# Iterar sobre los documentos y acceder a sus datos

valorizaciones={}
for doc in docs:
    print(f'ID del documento: {doc.id}')
    print(f'Datos del documento: {doc.to_dict()}')
    print("-" * 30)

    valorizaciones[doc.id]=doc.to_dict()

pprint.pprint(valorizaciones)

ID del documento: 4
Datos del documento: {'cargas_trabajo_contratista_corregido': {'MR301': 1999.875, 'MR201': 864.3035, 'MR202': 4.079, 'MR601': 43.003, 'MR101': 1.71, 'MR401': 16.168}, 'cargas_trabajo_contratista': {'MR301': 3600.0, 'MR201': 2720.0, 'MR202': 6.0, 'MR601': 64.5, 'MR101': 0.86, 'MR401': 5}, 'avance': {'porcentaje_ejecutado_acumulado_vigente_mes': 0.0591530651200461, 'porcentaje_programado_vigente_mes': 0.06666666666666667, 'porcentaje_programado_acumulado_vigente_mes': 0.06666666666666667, 'porcentaje_ejecutado_vigente_mes': 0.0591530651200461}}
------------------------------
ID del documento: 5
Datos del documento: {'cargas_trabajo_contratista_inicial': {'MR301': 1999.88, 'MR103': 13.33, 'MR201': 5146.67, 'MR202': 5.1, 'VP101': 242.5, 'MR101': 0.86, 'VP102': 112}, 'cargas_trabajo_contratista_corregido': {'MR301': 1999.88, 'MR103': 13.33, 'MR201': 5146.67, 'MR202': 5.1, 'VP101': 242.474, 'MR101': 0.86, 'VP102': 112}}
------------------------------
{'4': {'avance': {'po

## Cálculos

### Recalculo de precios unitarios

In [24]:
# Crear DataFrame usando pd.DataFrame.from_dict() con orient='index'
df_costos_expediente = pd.DataFrame.from_dict(
    costos_expediente, orient="index"
)

# Resetear el índice para convertir las claves MR en una columna
df_costos_expediente = df_costos_expediente.reset_index().rename(
    columns={"index": "codigo_MR"}
)

In [25]:
# Agregar columna parcial
df_costos_expediente["parcial"] = (
    df_costos_expediente["precio_unitario"]
    * df_costos_expediente["carga_trabajo"]
)
df_costos_expediente.head()

,codigo_MR,carga_trabajo,precio_unitario,parcial
0,MR301,15999.00,0.17,2719.8300
1,MR103,53.33,19.07,1017.0031
2,MR401,64.67,13.28,858.8176
3,MR104,85.33,21.18,1807.2894
4,MR601,258.02,2.44,629.5688


In [26]:
costo_directo = df_costos_expediente["parcial"].sum()
print("costo_directo", costo_directo)

costo_directo 152109.8165


In [27]:
costo_total_expediente = calculadora_costo_total(costo_directo,gastos_generales_expediente)
print(costo_total_expediente["total"])

206413.02


In [28]:
df_costos_expediente["precio_unitario_actualizado"] = (
    df_costos_expediente["precio_unitario"]
    * (monto_contrato / costo_total_expediente["total"])
)

df_costos_expediente["parcial_actualizado"] = (
    df_costos_expediente["precio_unitario_actualizado"]
    * df_costos_expediente["carga_trabajo"]
)

# Calcular el total
costo_directo_actualizado = df_costos_expediente["parcial_actualizado"].sum()

print(f"El costo directo es: {costo_directo_actualizado}")

El costo directo es: 121687.84140928515


In [29]:
df_costos_expediente.head(10)

,codigo_MR,carga_trabajo,precio_unitario,parcial,precio_unitario_actualizado,parcial_actualizado
0,MR301,15999.00,0.17,2719.8300,0.136000,2175.863789
1,MR103,53.33,19.07,1017.0031,15.255999,813.602401
2,MR401,64.67,13.28,858.8176,10.623999,687.054013
3,MR104,85.33,21.18,1807.2894,16.943998,1445.831380
4,MR601,258.02,2.44,629.5688,1.952000,503.654991
5,MR202,34.67,95.34,3305.4378,76.271993,2644.349984
6,MR101,10.28,317.80,3266.9840,254.239975,2613.586947
7,VP101,1512.58,47.01,71106.3858,37.607996,56885.103128
8,MR201,32350.00,0.53,17145.5000,0.424000,13716.398671
9,VP102,700.00,71.79,50253.0000,57.431994,40202.396105


In [30]:
dict_precios_unitarios_actualizados = dict(
    zip(
        df_costos_expediente["codigo_MR"],
        df_costos_expediente["precio_unitario_actualizado"],
    )
)

pprint.pprint(dict_precios_unitarios_actualizados)

{'MR101': 254.23997536589505,
 'MR103': 15.255998521798674,
 'MR104': 16.9439983582431,
 'MR201': 0.4239999589173203,
 'MR202': 76.27199260976852,
 'MR301': 0.1359999868225367,
 'MR401': 10.623998970607571,
 'MR601': 1.9519998108646441,
 'VP101': 37.60799635604382,
 'VP102': 57.431994435234756}


#### Cálculo de pago de acuerdo a cargas de trabajo

In [31]:
pago_costo_directo_parciales_contratista = calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo_expediente
)
pprint.pprint(pago_costo_directo_parciales_contratista)

{'MR101': 2613.586946761401,
 'MR103': 813.6024011675232,
 'MR104': 1445.8313799088837,
 'MR201': 13716.398670975312,
 'MR202': 2644.349983780675,
 'MR301': 2175.8637891737644,
 'MR401': 687.0540134291916,
 'MR601': 503.65499119929547,
 'VP101': 56885.10312822476,
 'VP102': 40202.39610466433}


In [32]:
df_pago_costo_directo_parciales_contratista = pd.DataFrame.from_dict(
    pago_costo_directo_parciales_contratista, orient="index"
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.reset_index().rename(
        columns={"index": "codigo_MR"}
    )
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.rename(columns={0: "monto_pago"})
)

df_pago_costo_directo_parciales_contratista

,codigo_MR,monto_pago
0,MR301,2175.863789
1,MR103,813.602401
2,MR401,687.054013
3,MR104,1445.831380
4,MR601,503.654991
5,MR202,2644.349984
6,MR101,2613.586947
7,VP101,56885.103128
8,MR201,13716.398671
9,VP102,40202.396105


In [33]:
sumar_valores = lambda d: sum(d.values())

In [34]:
pago_costo_directo_contratista = sumar_valores(pago_costo_directo_parciales_contratista)
print(pago_costo_directo_contratista)

121687.84140928513


In [35]:
pago_costo_total_contratista = calculadora_costo_total(pago_costo_directo_contratista,gastos_generales_expediente)
print(pago_costo_total_contratista["total"])

165130.39


#### Validación

In [36]:
diferencia_costos = abs(
    pago_costo_total_contratista["total"]
    - monto_contrato
)

pprint.pprint(
    {
        "diferencia_costos": diferencia_costos,
        "contrato": monto_contrato,
        "costos_unitarios": pago_costo_total_contratista["total"],
    }
)

if diferencia_costos > 1:
    raise ValueError(
        "La valorizacion de las actividades presentadas por el contratista no es coherente con lo programado"
    )

{'contrato': 165130.4,
 'costos_unitarios': 165130.39,
 'diferencia_costos': 0.009999999980209395}


#### Cálculo de cargas de trabajo ejecutadas del contratista

##### Mes en curso

In [37]:
cargas_trabajo_contratista_current_month=valorizaciones[str(mes_en_analisis)]['cargas_trabajo_contratista_corregido']
pprint.pprint(cargas_trabajo_contratista_current_month)

{'MR101': 0.86,
 'MR103': 13.33,
 'MR201': 5146.67,
 'MR202': 5.1,
 'MR301': 1999.88,
 'VP101': 242.474,
 'VP102': 112}


##### Acumulado

In [38]:
cargas_trabajo_ejecutadas_acumuladas={}
for v in valorizaciones:
    if(int(v)<=mes_en_analisis):
        cargas_trabajo_ejecutadas_acumuladas=sumar_dicts(cargas_trabajo_ejecutadas_acumuladas,valorizaciones[v]['cargas_trabajo_contratista_corregido'])

pprint.pprint(cargas_trabajo_ejecutadas_acumuladas)

{'MR101': 2.57,
 'MR103': 13.33,
 'MR201': 6010.9735,
 'MR202': 9.178999999999998,
 'MR301': 3999.755,
 'MR401': 16.168,
 'MR601': 43.003,
 'VP101': 242.474,
 'VP102': 112}


#### Cálculo de cargas de trabajo programadas

##### Mes en curso

In [39]:
cargas_trabajo_programadas_current_month = cargas_trabajo_programadas_anualmente[f"2025-{mes_en_analisis:02d}"].to_dict()
pprint.pprint(cargas_trabajo_programadas_current_month)

{'MR101': 0.856666667,
 'MR102': 0.0,
 'MR103': 13.3325,
 'MR104': 0.0,
 'MR111': 0.0,
 'MR112': 0.0,
 'MR201': 5146.590909,
 'MR202': 5.098529412,
 'MR203': 0.0,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 0.0,
 'MR301': 1999.875,
 'MR401': 0.0,
 'MR501': 0.0,
 'MR601': 0.0,
 'MR701': 0.0,
 'MR702': 0.0,
 'VP101': 242.5,
 'VP102': 112.0}


##### Acumulado

In [40]:
cargas_trabajo_programadas_anualmente.drop(columns=['TOTAL'], inplace=True)

In [41]:
# Asumiendo que tu dataframe se llama 'df'
cargas_trabajo_programadas_anualmente_dict = {}

# Iterar por las columnas del dataframe (excluyendo la columna 'codigo' si existe)
for columna in cargas_trabajo_programadas_anualmente.columns:
    if columna != 'codigo':  # Excluir la columna de códigos
        # Extraer el mes de la columna (formato "2025-MM")
        mes = int(columna.split('-')[1])  # Esto convierte "04" a 4, "12" a 12
        
        # Crear el diccionario interno para este mes
        cargas_trabajo_programadas_anualmente_dict[mes] = {}
        
        # Llenar el diccionario con los códigos y valores
        for codigo in cargas_trabajo_programadas_anualmente.index:  # Asumiendo que 'codigo' es el índice
            cargas_trabajo_programadas_anualmente_dict[mes][codigo] = float(cargas_trabajo_programadas_anualmente.loc[codigo, columna])

pprint.pprint(cargas_trabajo_programadas_anualmente_dict)

{4: {'MR101': 1.713333333,
     'MR102': 0.0,
     'MR103': 0.0,
     'MR104': 0.0,
     'MR111': 0.0,
     'MR112': 0.0,
     'MR201': 862.344,
     'MR202': 4.078823529,
     'MR203': 0.0,
     'MR204': 0.0,
     'MR205': 0.0,
     'MR206': 0.0,
     'MR301': 1999.875,
     'MR401': 16.1675,
     'MR501': 0.0,
     'MR601': 43.00333333,
     'MR701': 0.0,
     'MR702': 0.0,
     'VP101': 0.0,
     'VP102': 0.0},
 5: {'MR101': 0.856666667,
     'MR102': 0.0,
     'MR103': 13.3325,
     'MR104': 0.0,
     'MR111': 0.0,
     'MR112': 0.0,
     'MR201': 5146.590909,
     'MR202': 5.098529412,
     'MR203': 0.0,
     'MR204': 0.0,
     'MR205': 0.0,
     'MR206': 0.0,
     'MR301': 1999.875,
     'MR401': 0.0,
     'MR501': 0.0,
     'MR601': 0.0,
     'MR701': 0.0,
     'MR702': 0.0,
     'VP101': 242.5,
     'VP102': 112.0},
 6: {'MR101': 0.856666667,
     'MR102': 0.0,
     'MR103': 13.3325,
     'MR104': 14.22166667,
     'MR111': 0.0,
     'MR112': 0.0,
     'MR201': 1470.454545,
   

In [42]:
cargas_trabajo_programadas_acumuladas={}

for v in cargas_trabajo_programadas_anualmente_dict:
    if(int(v)<=mes_en_analisis):
        cargas_trabajo_programadas_acumuladas=sumar_dicts(cargas_trabajo_programadas_acumuladas,cargas_trabajo_programadas_anualmente_dict[v])

pprint.pprint(cargas_trabajo_programadas_acumuladas)

{'MR101': 2.5700000000000003,
 'MR102': 0.0,
 'MR103': 13.3325,
 'MR104': 0.0,
 'MR111': 0.0,
 'MR112': 0.0,
 'MR201': 6008.934909,
 'MR202': 9.177352941,
 'MR203': 0.0,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 0.0,
 'MR301': 3999.75,
 'MR401': 16.1675,
 'MR501': 0.0,
 'MR601': 43.00333333,
 'MR701': 0.0,
 'MR702': 0.0,
 'VP101': 242.5,
 'VP102': 112.0}


#### Avance valorizado ejecutado

##### Mensual

In [43]:
avance_valorizado_ejecutado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_contratista_current_month)

pprint.pprint(avance_valorizado_ejecutado)

{'MR101': 218.64637881466973,
 'MR103': 203.3624602955763,
 'MR104': 0.0,
 'MR201': 2182.187868561005,
 'MR202': 388.98716230981944,
 'MR301': 271.9836536466547,
 'MR401': 0.0,
 'MR601': 0.0,
 'VP101': 9118.96130843537,
 'VP102': 6432.383376746293}


In [44]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_ejecutado=sumar_valores(avance_valorizado_ejecutado)
print(costo_directo_avance_valorizado_ejecutado)

18816.51220880939


In [45]:
costo_total_avance_valorizado_ejecutado = calculadora_costo_total(costo_directo_avance_valorizado_ejecutado,gastos_generales_expediente)
print(costo_total_avance_valorizado_ejecutado["total"])

25534.01


##### Acumulado

In [46]:
avance_valorizado_ejecutado_acumulado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_ejecutadas_acumuladas)

pprint.pprint(avance_valorizado_ejecutado_acumulado)

{'MR101': 653.3967366903503,
 'MR103': 203.3624602955763,
 'MR104': 0.0,
 'MR201': 2548.652517053101,
 'MR202': 700.1006201650652,
 'MR301': 543.9666272933753,
 'MR401': 171.76881535678322,
 'MR601': 83.94184786661229,
 'VP101': 9118.96130843537,
 'VP102': 6432.383376746293}


In [47]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_ejecutado_acumulado=sumar_valores(avance_valorizado_ejecutado_acumulado)
print(costo_directo_avance_valorizado_ejecutado_acumulado)

20456.534309902527


In [48]:
costo_total_avance_valorizado_ejecutado_acumulado = calculadora_costo_total(costo_directo_avance_valorizado_ejecutado_acumulado,gastos_generales_expediente)
print(costo_total_avance_valorizado_ejecutado_acumulado["total"])

27759.51


### Avance valorizado programado

#### Mensual

In [49]:
avance_valorizado_programado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_programadas_current_month)

pprint.pprint(avance_valorizado_ejecutado)

{'MR101': 218.64637881466973,
 'MR103': 203.3624602955763,
 'MR104': 0.0,
 'MR201': 2182.187868561005,
 'MR202': 388.98716230981944,
 'MR301': 271.9836536466547,
 'MR401': 0.0,
 'MR601': 0.0,
 'VP101': 9118.96130843537,
 'VP102': 6432.383376746293}


In [50]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_programado=sumar_valores(avance_valorizado_programado)
print(costo_directo_avance_valorizado_programado)

18816.53431095339


In [51]:
costo_total_avance_valorizado_programado = calculadora_costo_total(costo_directo_avance_valorizado_programado,gastos_generales_expediente)
print(costo_total_avance_valorizado_programado["total"])

25534.03


#### Acumulado

In [52]:
avance_valorizado_programado_acumulado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_programadas_acumuladas)

pprint.pprint(avance_valorizado_programado_acumulado)

{'MR101': 653.3967366903504,
 'MR103': 203.4006002918808,
 'MR104': 0.0,
 'MR201': 2547.7881545528517,
 'MR202': 699.9749956931894,
 'MR301': 543.9659472934411,
 'MR401': 171.7635033572979,
 'MR601': 83.94249852670924,
 'VP101': 9119.939116340627,
 'VP102': 6432.383376746293}


In [53]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_programado_acumulado=sumar_valores(avance_valorizado_programado_acumulado)
print(costo_directo_avance_valorizado_programado_acumulado)

20456.554929492642


In [54]:
costo_total_avance_valorizado_programado_acumulado = calculadora_costo_total(costo_directo_avance_valorizado_programado_acumulado,gastos_generales_expediente)
print(costo_total_avance_valorizado_programado_acumulado["total"])

27759.55


### Resultados

In [55]:
porcentaje_avance_ejecutado = (
    costo_total_avance_valorizado_ejecutado["total"] / monto_contrato
)
porcentaje_avance_ejecutado_acumulado = (
    costo_total_avance_valorizado_ejecutado_acumulado["total"] / monto_contrato
)
porcentaje_avance_programado = (
    costo_total_avance_valorizado_programado["total"] / monto_contrato
)
porcentaje_avance_programado_acumulado = (
    costo_total_avance_valorizado_programado_acumulado["total"] / monto_contrato
)

print(
    f"{porcentaje_avance_ejecutado=}\n{porcentaje_avance_ejecutado_acumulado=}\n{porcentaje_avance_programado=}\n{porcentaje_avance_programado_acumulado=}"
)

porcentaje_avance_ejecutado=0.1546293716965501
porcentaje_avance_ejecutado_acumulado=0.16810659939054226
porcentaje_avance_programado=0.15462949281295268
porcentaje_avance_programado_acumulado=0.16810684162334738


In [56]:
data_to_print={
    "porcentaje_avance_ejecutado":porcentaje_avance_ejecutado,
    "porcentaje_avance_ejecutado_acumulado":porcentaje_avance_ejecutado_acumulado,
    "porcentaje_avance_programado":porcentaje_avance_programado,
    "porcentaje_avance_programado_acumulado":porcentaje_avance_programado_acumulado
}

In [57]:
ruta_directorio = os.path.join("output", proyecto, str(mes_en_analisis))
ruta_archivo = os.path.join(ruta_directorio, "cargas_trabajo.json")

# Crear el directorio si no existe
os.makedirs(ruta_directorio, exist_ok=True)
# Guardar el diccionario en un archivo JSON

with open(ruta_archivo, "w", encoding="utf-8") as archivo_json:
    json.dump(data_to_print, archivo_json, ensure_ascii=False, indent=4)